# JS05 - Graph Traversals

## Overview

A graph consists of vertices and edges. Vertices are stored as documents in vertex collections, and edges are stored as documents in edge collections. The collections used in a graph and their relations are specified with edge definitions.

This tutorial will build a simple application for people to find new friends called MacroFriends, and then use graph traversals to see who would like to meet whom.

## Pre-requisite

Let's assume: 
- You have already made a tenant account, and have a username and password
- You have installed the jsC8 drivers as explained in section 01
- You have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## 1. Importing Libraries & Define Variables

In [1]:
// Variables - Queries
const global_url = "https://gdn.paas.macrometa.io";
const token = ""; // <-- Token goes here if applicable
const apiKey = ""; // <-- apiKey goes here if applicable
const email = "email"; // <-- Email goes here
const password = "password"; // <-- password goes here
const geo_fabric = "_system";
const collection_friends = "Friends";
const collection_likes = "Likes";

const collection_graph = "Macro_Friends_Graph";
const read_friends = "FOR friends IN Friends RETURN friends";
const read_likes = "FOR likes IN Likes RETURN like";

const macrofriends_client="SanJoseLou";
const macrofriends_client2="StocktonPeria";

const graph_traversal1 = `FOR c IN Friends FILTER c.macrofriend_user_name == \"${macrofriends_client}\" FOR v IN 1..1 INBOUND c Likes RETURN v`;
const graph_traversal2 = `FOR c IN Friends FILTER c.macrofriend_user_name == \"${macrofriends_client}\" FOR v IN 1..1 OUTBOUND c Likes RETURN v`;

## 2. Connecting to GDN

In [2]:
const jsc8 = require("jsc8");

// Simple Way
// const client = new jsc8({
//   url: global_url,
//   token: token,
//   fabricName: geo_fabric,
// });

// ----- OR -----
// const client = new jsc8({url: global_url, apiKey: apiKey, fabricName: geo_fabric});

// To use advanced options
const client = new jsc8(global_url);
client
  .login(email, password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));


## 3. Creating Collections

In [3]:
async function createCollection() {
  console.log("\n 2. CREATE_COLLECTION");

  try {
    console.log(
      `Creating the CREATE_FRIENDS_VERTEX_COLLECTION ${collection_friends}...`
    );
    let isCollectionExists = await client.hasCollection(collection_friends);
    if (!isCollectionExists) {
      await client.createCollection(collection_friends);
    }

    console.log(
      `Creating the CREATE_LIKES_EDGE_COLLECTION ${collection_likes}...`
    );
    isCollectionExists = await client.hasCollection(collection_likes);
    if (!isCollectionExists) {
      await client.createCollection(collection_likes, {}, true);
    }
  } catch (e) {
    console.log("Collection creation did not succeed due to " + e.message);
  }
}

createCollection();


## 4. Inserting Data - "Friends"

In [4]:
let documents = [
  {
    full_name: "Kiel Dummer",
    first_name: "Kiel",
    last_name: "Dummer",
    city: "Burbank",
    state: "CA",
    address: "40317 5th Crossing",
    macrofriend_user_name: "BurbankKeil",
    _key: "BurbankKeil",
  },
  {
    full_name: "Pernell Winspare",
    first_name: "Pernell",
    last_name: "Winspare",
    city: "San Diego",
    state: "CA",
    address: "596 Packers Pass",
    macrofriend_user_name: "SanDiegoPernell",
    _key: "SanDiegoPernell",
  },
  {
    full_name: "Ava Kermath",
    first_name: "Ava",
    last_name: "Kermath",
    city: "Berkeley",
    state: "CA",
    address: "2 Doe Crossing Junction",
    macrofriend_user_name: "BerkeleyAva",
    _key: "BerkeleyAva",
  },
  {
    full_name: "Tremain McGrah",
    first_name: "Tremain",
    last_name: "McGrah",
    city: "Torrance",
    state: "CA",
    address: "079 Russell Street",
    macrofriend_user_name: "TorranceTremain",
    _key: "TorranceTremain",
  },
  {
    full_name: "Vidovik Boddam",
    first_name: "Vidovik",
    last_name: "Boddam",
    city: "Los Angeles",
    state: "CA",
    address: "3 Brentwood Crossing",
    macrofriend_user_name: "LosAngelesVidovik",
    _key: "LosAngelesVidovik",
  },
  {
    full_name: "Oralie Goward",
    first_name: "Oralie",
    last_name: "Goward",
    city: "Los Angeles",
    state: "CA",
    address: "922 Columbus Park",
    macrofriend_user_name: "LosAngelesOralie",
    _key: "LosAngelesOralie",
  },
  {
    full_name: "Lou Feaveer",
    first_name: "Lou",
    last_name: "Feaveer",
    city: "San Jose",
    state: "CA",
    address: "55223 Hooker Crossing",
    macrofriend_user_name: "SanJoseLou",
    _key: "SanJoseLou",
  },
  {
    full_name: "Peria King",
    first_name: "Peria",
    last_name: "King",
    city: "Stockton",
    state: "CA",
    address: "8 Troy Plaza",
    macrofriend_user_name: "StocktonPeria",
    _key: "StocktonPeria",
  },
];

async function insertDocumentMany() {
  console.log(`\n 3a. INSERT_FRIENDS_DATA in region ${global_url}`);
  await client.insertDocumentMany(collection_friends, documents);
}

insertDocumentMany();


## 5. Inserting Data - "Who Likes Whom"

In [5]:
documents = [
  {
    macrofriend_user_name: "BurbankKeil",
    friends_liked: "SanJoseLou",
    _from: "Friends/BurbankKeil",
    _to: "Friends/SanJoseLou",
    liked_date: "1/4/2020",
    liked_time: "13:33",
    liked_amount: 30,
  },
  {
    macrofriend_user_name: "SanJoseLou",
    friends_liked: "StocktonPeria",
    _from: "Friends/SanJoseLou",
    _to: "Friends/StocktonPeria",
    liked_date: "1/28/2020",
    liked_time: "3:02",
    liked_amount: 18,
  },
  {
    macrofriend_user_name: "BerkeleyAva",
    friends_liked: "StocktonPeria",
    _from: "Friends/BerkeleyAva",
    _to: "Friends/StocktonPeria",
    liked_date: "1/28/2020",
    liked_time: "3:02",
    liked_amount: 18,
  },
];

async function insertDocumentMany() {
  console.log(`\n 3a. INSERT_LIKES_DATA in region ${global_url}`);
  await client.insertDocumentMany(collection_likes, documents);
}

insertDocumentMany();


## 6. Creating a Graph

In [6]:
async function createGraph() {
  console.log(`\n 4. CREATE_GRAPH ... FriendFinder_graph region ${global_url}`);

  await client.createGraph(collection_graph, {
    edgeDefinitions: [
      {
        collection: collection_likes,
        from: [collection_friends],
        to: [collection_friends],
      },
    ],
  });
  console.log("Graph created");
}

createGraph().catch((err) =>
  console.log("Error while creating graph :", err.message)
);


## 7. Execute a Traversal - "Find Friends who who would like to meet Lou?"

In [7]:
async function readData() {
  console.log(
    `5a. GRAPH_TRAVERSAL: Find Friends who would like to meet: ${macrofriends_client}`
  );
  let result = await client.executeQuery(graph_traversal1);
  console.log(result);
}

readData().catch((err) =>
  console.log("Error while execute a traversal :", err.message)
);


## 8. Execute a Traversal - "“Find Friends who Lou would like to meet?"

In [8]:
async function readData() {
  console.log(
    `5b. GRAPH_TRAVERSAL: Find Friends who were liked by Lou: ${macrofriends_client2}`
  );
  result = await client.executeQuery(graph_traversal2);
  console.log(result);
}

readData().catch((err) =>
  console.log("Error while execute a traversal :", err.message)
);


## 9. Delete Data and Collections

In [9]:
async function deleteData() {
  console.log(" 6. DELETE_DATA");
  await client.deleteGraph(collection_graph, true);
  console.log("Collection and Graph deleted");
}

deleteData().catch((err) =>
  console.log("Error while deleting collection and graphs:", err.message)
);


## Section Completed!

TBC...